# **What's Ur EDA? | Exploratory Data Analysis I**

Hello! Welcome to your first ipynb file for this workshop! The language we are using is Python. Here, we will cover data inspection and data cleaning techniques.
After this segment, you will learn how to:
1. Read a csv file into the Python environment
2. Investigate the features of your dataset
3. Deal with duplicated values, inconsistent labelling, missing and erroneous values
4. Generate a Profile Report with YData as an alternative

Please ensure that you have downloaded "employee_dirty.csv" and uploaded them (right click) on the side under "Files". You can use the "Table of Contents" tab at the side to help with navigation.

The original csv file can be found from https://www.kaggle.com/datasets/tawfikelmetwally/employee-dataset.

Let's begin!

Import relevant libraries

In [ ]:
import pandas as pd  # used for data manipulation & analysis
import numpy as np   # super fast calculator

Import dataset

In [ ]:
df = pd.read_csv("employee_dirty.csv") # Check if the first row is a header. If not, header=None

# Data Inspection

Before we begin on data cleaning, we will run some codes to understand the features of our dataset. df.head() provides a quick overview of your data's structure and content.

To customise the number of rows you wish to see, you can add a numerical argument into the parentheses.
For instance, df.head(10) gives the first 10 rows of your dataframe.

In [ ]:
# Shows the first 5 rows of dataframe & column headers
df.head()

Alternatively, you may consider looking at the last few rows.

In [ ]:
# Shows the last 5 rows
df.tail()

## Shape

We may want to look at the number of rows and columns in the dataframe.

The result (x, y) indicates the following: (no. of rows, no. of columns)

In [ ]:
# No. of rows & columns
df.shape  # attributes have no ()

## Data Types

We may also wish to look at the data types of our columns to identify if they are numeric, strings or dates.
- Numeric: int64, float64
- String: object (typically strings)
- Date/Time: datetime64



In [ ]:
# Data types of each column
df.dtypes

# Data Summary

To obtain a more comprehensive (and useful) summary of the dataframe, we can use df.info() which gives us
1. No. of entries / rows
2. Column Information (column names, non-null count, data type)
3. Memory Usage of the Dataframe

From the non-null count, we can infer if there are missing values. Non-Null Count column tells you how many rows in that column **do not** contain missing values (NaN -- Not a Number or None)

In [ ]:
# Concise summary of dataframe
df.info()

# Descriptive Statistics
df.describe() generates descriptive statistics for your dataframe.
- For numerical columns, it summarises the count, mean, measure of position (quartiles), and measures of variability (standard deviation, range from min/max values).

- For text columns, it summarises the count, number of unique values (categories), the most frequent value and its frequency.

In [ ]:
df.describe(include="all") # all variables -- include="all" is the default argument

# df.describe(include="number") # Only variables with integers/floats
# df.describe(include="object") # Only categorical variables with string data

## Value Counts

df.describe() does not do a very good job at revealing useful information about your text columns.

.value_counts() solves this by giving a count of each category within the specified column, making it easier to derive the same information as df.describe() but in greater detail.

In [ ]:
print(df["Education"].value_counts())
print() # leaves an empty line

print(df["City"].value_counts())
print()

print(df["Gender"].value_counts())
print()

print(df["EverBenched"].value_counts())
print()

# Response Variable
print(df['LeaveOrNot'].value_counts())

## Unique Values

**df.nunique()** provides the count of distinct values in each column.

- Easier to spot mistakes
  - e.g. We expect “Gender” to have 2 unique values but we see 4

- Useful for discovering unexpected duplicates
  - e.g. We expect 100k unique data points in your dataset but we only see 99k unique values

In [ ]:
# No. of unique elements in each column
df.nunique()

.unique() can be used to check the unique elements in each column.
- One advantage is that you can now see NaN values which .value_counts() could not show.

To perform the same task as what we have done for "Education", we can use a "for loop" to execute the same block of code repeatedly.

In [ ]:
# Check unique elements for a specific column
print(df["Education"].unique())
print()

# Check unique elements for all columns
for col in df.columns:  # runs lines 7 - 9 for all column names
  print(col)
  print(df[col].unique())
  print()

# Data Cleaning

From the inspection we have done, there are a few things that stand out:
1. Age has NaN and negative values.
2. Gender has two ways of classifying each gender.
3. ExperienceInCurrentDomain has an unrealistic number (erroneous value).
4. Education has NaN.

We will take note of this and clean the dataset.

## Duplicated Values

We will need to check for duplicates within our data as a first step. Repeated data are redundant and will hold more weight within the dataset so they can skew the analysis. This leads to problems like
1. Inflated / deflated counts of certain values such as mean, std
2. Incorrect visual representations of data
    - Customer purchase recorded twice --> artificially higher transaction count --> flawed image of business performance
3. Overfitting in machine learning
    - Some data points appear more common than they actually are so the model's parameters are overly tuned to these particular points instead of learning the general pattern of the data

In [ ]:
# Shows the duplicated rows
df[df.duplicated()]

# The default argument is keep="first" which only shows the second and subsequent duplicates
# keep=False will include the first one to give the complete set of duplicates
# df[df.duplicated(keep=False)]

From the above, we get a count of 1544 duplicated rows (excessive) so we use df.drop_duplicates() to remove them.

.copy() tells Pandas explicitly to make a unique copy of the old dataframe i.e. changing values in the new dataframe does not change values in the old dataframe.

In [ ]:
# Removes duplicates and keeps unique rows
df_clean = df.drop_duplicates().copy()  # creates a new dataframe without the duplicates
df_clean.info()

The original index labels still remain so RangeIndex has values from 0 to 4651. We want our row indexes to not have jumps in the sequence.

In [ ]:
# Reset indexes
df_clean.reset_index(drop=True, inplace=True) # drop=True ensures that the old index is not added as a column by itself
df_clean.info()

In [ ]:
# Check if there's still duplicated values
print(df_clean.duplicated().sum())

## Inconsistent Labelling

In [ ]:
print(df_clean["Gender"].value_counts())

We can use .replace(old, new) to find all instances of "old" to replace with "new".

.loc is a label-based indexer that allows you to select data based on the row and column labels / names.

We can use : to select all the rows in the dataframe.

In [ ]:
# Assign the updated column (right) back into the original column (left)
df_clean.loc[:, "Gender"] = df_clean["Gender"].replace("m", "Male")
df_clean.loc[:, "Gender"] = df_clean["Gender"].replace("F", "Female")
print(df_clean["Gender"].value_counts())

## Erroneous Values & Missing Values

These 2 types of data quality issues involve changing values. There are 3 main methods to address these problems.
1. Delete rows with missing values
2. Impute using mean/median/mode (Univariate -- use only info from the same column to fill in missing values)
3. Advanced imputation methods like KNN imputation (Multivariate -- use info from other columns to fill in missing values)

Imputation --> assign values by inference

### Missing Value Check

We need to check if there are any missing values within each column.

df.isna() checks if the column values are empty or not.
- Empty --> True
- Not empty --> False
- Taking the sum for each column = Summing the True's = Counting the empty cells

In [ ]:
# Counts the missing values
print(df_clean.isna().sum())   # df.isna().any() gives True / False to indicate if the column has empty values or not

### Imputing for Categorical Variables

In [ ]:
print(df_clean["Education"].value_counts())
print(f"Number of NaN values:{(df_clean['Education']).isna().sum()}" )  # f"..." is called f-string.
print()

# Also, note that .count() only considers non NaN values
# 2213 + 704 + 161 = 3078 (does not include the 31 NaN values!)
print(f"Total Count of rows: {df_clean['Education'].count()}")

Now that we have checked that there are 31 missing values, we can proceed to impute the missing values. There are two ways to do this:
1. Impute the most frequent value
2. Impute the value "Missing" such that Python treats it as a separate category

Since there are few missing values relative to the size of the dataset, we will choose to impute the most frequent value.

In [ ]:
from sklearn.impute import SimpleImputer  # sklearn library supports imputation
imputer = SimpleImputer(strategy="most_frequent")
df_clean["Education"] = imputer.fit_transform(df_clean[["Education"]]).ravel()  # Note the two square brackets

# .ravel() flattens the 2D numPy array to 1D --> (no. of rows, )

All NaN values have been changed to "Bachelors" as shown in the code below.

In [ ]:
print(df_clean["Education"].value_counts())
print(f"Number of NaN values: {df_clean['Education'].isna().sum()}")

### Imputing for Quantitative Variables

For quantitative variables, some data quality issues we may face are missing values, and negative values. In particular for missing values, there are numerous ways to deal with them for quantitative variables. What is important here is to choose a method based on how the data is missing (Missing Completely At Random, Missing At Random, Missing Not At Random), the percentage of missing values and the type of missing value.

Some ways to deal with missing values are:
1. Delete the rows with the missing values
2. Use mean/mode/median imputation
3. Use advanced imputation techniques like KNN imputation

In general, it is a good idea to start with columns with few or too many missing values.



#### ExperienceInCurrentDomain

In [ ]:
print(df_clean["ExperienceInCurrentDomain"].value_counts())

From the investigation above, we can see that there are erroneous values in ExperienceInCurrentDomain.

The most likely reason for this is a typo -- it could have been 20 or 2.
Since 200 can skew the mean significantly, we will impute values based on the mode/median instead.

In [ ]:
df_clean["ExperienceInCurrentDomain"] = df_clean["ExperienceInCurrentDomain"].replace(200, np.nan)

from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="most_frequent")
df_clean["ExperienceInCurrentDomain"] = imputer.fit_transform(df_clean[["ExperienceInCurrentDomain"]]).ravel()

In [ ]:
print(df_clean["ExperienceInCurrentDomain"].value_counts())

#### Age
In the column Age, we have negative values and NaN.

In [ ]:
print(df_clean["Age"].value_counts())
print()
print(df_clean["Age"].isna().sum())

It is likely that the negative values were simply recorded wrongly. The assumption here is that the errors are not systematic (for instance they are not occurring only for certain data collectors).

For this, we have a larger number of missing data (169). While it's still somewhat small compared to the size of the dataset (6.56%), it is a lot larger than what we have been dealing with. To be cautious, we can consider more sophisticated imputation methods.

kNN imputation finds the closest data points (neighbours) based on available features and uses their values to estimate the missing value.

In [ ]:
# Step 1: Change negative values to NaN
df_clean["Age"] = df_clean["Age"].replace([-9.0, -1.0], np.nan)


# Step 2: OneHotEncoding
categorical_col = ['Education', 'City', 'Gender', 'EverBenched']
df_encoded = pd.get_dummies(df_clean, columns=categorical_col, drop_first=True)


# Step 3: Normalisation
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
col_to_normalise = ["Age", "ExperienceInCurrentDomain", "PaymentTier", "JoiningYear"]
df_encoded[col_to_normalise] = scaler.fit_transform(df_encoded[col_to_normalise])   # normalise quantitative variables


# Step 4: KNN Imputation
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=55)   # general rule of thumb = sqrt(N) where N is the no. of rows

df_for_imputation = df_encoded.drop(columns=["LeaveOrNot"])  # remove column
array_knn = knn_imputer.fit_transform(df_for_imputation)
df_clean_imputed = pd.DataFrame(array_knn, columns=df_for_imputation.columns)  # convert to DataFrame
df_clean_imputed["LeaveOrNot"] = df_encoded["LeaveOrNot"]   # add back column


# Step 5: Convert quantitative variables back to their original scale
# The scaler stored the original min/max values for each column
array_unscaled = scaler.inverse_transform(df_clean_imputed[col_to_normalise])   # reverses the scaling process

# We convert the array to a DataFrame
df_clean_unscaled = pd.DataFrame(array_unscaled, columns=col_to_normalise)


# Step 6: Round values to integers
df_clean_unscaled["Age"] = df_clean_unscaled["Age"].astype(int)


# Step 7: Update the "Age" column in the original DataFrame
df_clean["Age"] = df_clean_unscaled["Age"]

There are no more negative values!

In [ ]:
print(df_clean["Age"].isna().sum())
print(df_clean["Age"].value_counts())

## Final Checks

In [ ]:
df_clean.info()
for col in df_clean.columns:
  print(col)
  print(df_clean[col].unique())
  print()

We can now export the cleaned dataset.

In [ ]:
df_clean.to_csv("employee_cleaned.csv", index=False)  # Warning: you should have run *ALL* code blocks before running this

# YData Profile Report

In [ ]:
!pip install ydata_profiling
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Employee", explorative=True)
profile.to_file("your_report.html")

from google.colab import files
files.download('your_report.html')

In [ ]:
# For YData Report Verification
print(f"Duplicates from Python (including the first one): {df[df.duplicated(keep=False)].shape[0]}")
print(f"Duplicates from Python: {df[df.duplicated()].shape[0]}")
print(f"Duplicates from YData: {df[df.duplicated(keep=False)].shape[0] - df[df.duplicated()].shape[0]}")

2193-1544 = 649. Verified that YData duplicate count is correct!

# Miscellaneous
Code for generating dirty dataset

In [ ]:
'''
# Change to NaN for Age (Missing Values)
rows_to_change_nan = df.sample(n = 200, random_state=2040).index
df.loc[rows_to_change_nan, "Age"] = np.nan

# Change 2 in ExperienceInCurrentDomain to 200 (Erroneous Values)
df_filtered = df[df["ExperienceInCurrentDomain"] == 2]  # df with rows that have exp = 2 years
rows_to_change_2 = df_filtered.sample(n = 54, random_state=2040).index
df.loc[rows_to_change_2, "ExperienceInCurrentDomain"] = 200

# Change Male in Gender to m and Female in Gender to F (Inconsistent Labelling)
df_male = df[df["Gender"] == "Male"]
rows_to_change_male = df_male.sample(n = 378, random_state=2040).index
df.loc[rows_to_change_male, "Gender"] = "m"

df_female = df[df["Gender"] == "Female"]
rows_to_change_female = df_female.sample(n = 103, random_state=2040).index
df.loc[rows_to_change_female, "Gender"] = "F"

# Change 26 to -1 and -9 for Age (Negative Values)
df_young = df[df["Age"] == 26]
rows_to_change_young = df_young.sample(n = 43, random_state=2040).index
df.loc[rows_to_change_young, "Age"] = -1

rows_to_change_young2 = df_young.sample(n = 39, random_state=2040).index
df.loc[rows_to_change_young2, "Age"] = -9

# Change Bachelors to NaN for Education (Categorical Variables)
df_bachelors = df[df["Education"] == "Bachelors"]
rows_to_change_bachelors = df_bachelors.sample(n = 32, random_state=2040).index
df.loc[rows_to_change_bachelors, "Education"] = np.nan
'''

In [ ]:
# df.to_csv("employee_dirty.csv", index=False, na_rep="")